In [1]:
import cv2
import json
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [2]:
import cv2
import json
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix




tools = ['accendino', 'cacciavite', 'chiave', 'forbici', 'martello', 'metro', 'nastro', 'pappagallo', 'penna', 'spillatrice']
tp_tot, tn_tot, fp_tot, fn_tot = [], [], [], []


yolo_model = cv2.dnn.readNetFromDarknet("yolov3.cfg", "yolov3.weights")


for tool in tqdm(tools):

    with open(f"C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\{tool}\{tool}_gt.json", 'r') as f:
        ground_truth_file = json.load(f)

    for pos, el in enumerate(ground_truth_file):
        # Ground truth.
        x_pos = ground_truth_file[el]['regions'][0]['shape_attributes']['all_points_x']
        y_pos = ground_truth_file[el]['regions'][0]['shape_attributes']['all_points_y']
        vertices = np.array([[x_pos[i], y_pos[i]] for i in range(len(x_pos))])

        image = cv2.imread(f"C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\{tool}\{tool}_{pos+1}.png")

        if pos < 5:
            background_path = "C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\\background\\bianco.jpg"
        elif pos < 10:
            background_path = "C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\\background\\cemento.jpg"
        elif pos < 15:
            background_path = "C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\\background\\bancone.jpg"
        else:
            background_path = "C:/Users\mcm23\OneDrive\Desktop\GitHub\Object_Detection\images\\background\\coperta.jpg"


        a = 5
        # IMAGE PREPROCESSING
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


                # # # BACKGROUND SUBTRACTION
        background = cv2.imread(background_path)
        background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(background, gray)
        # # diff = cv2.GaussianBlur(diff, (5, 5), cv2.BORDER_DEFAULT)




        # gray = cv2.GaussianBlur(gray, (a, a), cv2.BORDER_DEFAULT)     # problem with cacciavite, toglie il gambo
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        eroded_gray = cv2.erode(diff, kernel, iterations=3)

        kernel = np.ones((3,3), np.uint8)
        dilated_img = cv2.dilate(eroded_gray, kernel, iterations=1)



        threshold, mask = cv2.threshold(dilated_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # cv2.imshow("Masked Image", mask)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()


        # # # BACKGROUND SUBTRACTION
        # background = cv2.imread(background_path)
        # background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
        # eroded = cv2.erode(background, kernel, iterations=3)
        # # background = cv2.GaussianBlur(background, (a, a), cv2.BORDER_DEFAULT)
        # diff = cv2.absdiff(eroded, eroded_gray)
        # # diff = cv2.GaussianBlur(diff, (5, 5), cv2.BORDER_DEFAULT)



        # cv2.imshow("Masked Image", diff)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()


        # edges = cv2.Canny(diff, 30, 100)
        # contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # cv2.drawContours(image, contours, -1, (0, 255, 0), 2)

        # cv2.imshow("Masked Image", image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()





        # thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)




        # # # applicare la soglia alla mappa di sfondo
        # threshold, mask = cv2.threshold(threshold, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

        # # Apply erosion to remove small patterns on the background
        # eroded = cv2.erode(mask, kernel, iterations=3)




        # masked_image = cv2.bitwise_and(image, image, mask=mask)

        # final = cv2.cvtColor(masked_image, cv2.COLOR_BGR2GRAY)


        # # Display the resulting image
        # cv2.imshow("Masked Image", eroded)
        # cv2.waitKey(0)









        # Applicare ground truth all'immagine.
        truth_mask = np.zeros_like(threshold)
        cv2.fillPoly(truth_mask, [vertices], 255)
        truth_mask = truth_mask.astype(bool).flatten()

        opening = mask.astype(bool)
        opening = np.array([not elem for elem in opening.flatten()])
        print(opening)

        # Confronto tra segmentazione e ground truth.
        ground_truth = truth_mask.flatten()
        predictions = [not elem for elem in opening.flatten()]
        tn, fp, fn, tp = confusion_matrix(ground_truth, predictions, labels=[True, False]).ravel()
        tp_tot.append(tp)
        tn_tot.append(tn)
        fp_tot.append(fp)
        fn_tot.append(fn)

  0%|          | 0/10 [00:00<?, ?it/s]

[ True  True  True ...  True  True  True]


ValueError: Found input variables with inconsistent numbers of samples: [1, 270400]

In [2]:
sum_tp, sum_tn, sum_fp, sum_fn = sum(tp_tot), sum(tn_tot), sum(fp_tot), sum(fn_tot) 

cf_matrix = np.array([[sum_tp/(sum_tp+sum_fp), sum_fp/(sum_tp+sum_fp)],
                       [sum_fn/(sum_tn+sum_fn), sum_tn/(sum_tn+sum_fn)]])

labels = ['Tool', 'Background']
sns.heatmap(cf_matrix, annot=True, cmap='viridis',  xticklabels=labels, yticklabels=labels, cbar=False, fmt=".1%")
plt.title("K-means (per clustering)")
plt.xlabel('Actual')
plt.ylabel('Predicted');

NameError: name 'tp_tot' is not defined

 10%|█         | 1/10 [00:31<04:47, 31.90s/it]